Loan word classification experiments

In [1]:
import sklearn as sk
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import pandas as pd 
import numpy as np 
import io
import requests
import csv
import json
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import pprint
import matplotlib.pyplot as plt
import random

In [2]:
import torch
from torch import nn
from torch import optim
from sklearn.model_selection import train_test_split

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
# device = 'cpu'
# print(device)

Using cuda device


In [4]:
pairs = None

with open('../language-pairs.json', 'r') as f:
    pairs = json.loads(f.read())
    print(pairs)
    
train_alldata = {}
test_alldata = {}

train_realdist = {}
test_realdist = {}

train_balanced = {}
test_balanced = {}

for pair in pairs:
    print(pair)
    L1 = pairs[pair]['target']['name']
    L2 = pairs[pair]['source']['name']

    # load datasets
    prefix = f'../Datasets/production_train_test/{L1}-{L2}'

    train_alldata[f'{L1}-{L2}'] = pd.read_csv(f'{prefix}/alldata/{L1}-{L2}-train_production_alldata.csv')
    test_alldata[f'{L1}-{L2}'] = pd.read_csv(f'{prefix}/alldata/{L1}-{L2}-test_production_alldata.csv')

    train_realdist[f'{L1}-{L2}'] = pd.read_csv(f'{prefix}/realdist/{L1}-{L2}-train_production_realdist.csv')
    test_realdist[f'{L1}-{L2}'] = pd.read_csv(f'{prefix}/realdist/{L1}-{L2}-test_production_realdist.csv')

    train_balanced[f'{L1}-{L2}'] = pd.read_csv(f'{prefix}/balanced/{L1}-{L2}-train_production_balanced.csv')
    test_balanced[f'{L1}-{L2}'] = pd.read_csv(f'{prefix}/balanced/{L1}-{L2}-test_production_balanced.csv')
    
    
features = ['Fast Levenshtein Distance Div Maxlen',
            'Dolgo Prime Distance Div Maxlen',
            'Feature Edit Distance Div Maxlen',
            'Hamming Feature Distance Div Maxlen',
            'Weighted Feature Distance Div Maxlen',
            'Partial Hamming Feature Distance Div Maxlen',
            'plain Levenshtein',
            'DNN_logits',
            'MBERT_cos_sim',
            'XLM_cos_sim'
           ]


# features = ['Fast Levenshtein Distance Div Maxlen',
#             'Dolgo Prime Distance Div Maxlen',
#             'Feature Edit Distance Div Maxlen',
#             'Hamming Feature Distance Div Maxlen',
#             'Weighted Feature Distance Div Maxlen',
#             'Partial Hamming Feature Distance Div Maxlen',
#             'plain Levenshtein',
             
#            ]




# features = [ 
#             'plain Levenshtein',
#             'DNN_logits',
#             'MBERT_cos_sim',
#             'XLM_cos_sim'
#            ]


# features = [ 
#             'plain Levenshtein',
             
#             'MBERT_cos_sim',
#             'XLM_cos_sim'
#            ]

# features = [ 
            
#                 'DNN_logits',
             
#             'MBERT_cos_sim',
#             'XLM_cos_sim'
#            ]
# features = [ 
#             'Partial Hamming Feature Distance Div Maxlen',
#             'plain Levenshtein',
#                 'DNN_logits',
             
#             'MBERT_cos_sim',
#             'XLM_cos_sim'
#            ]


labels = ['label_bin']

train_allpairs_alldata = pd.concat([train_alldata[pair] for pair in pairs])
x_train_allpairs_alldata = np.hstack([train_allpairs_alldata[features].values])
y_train_allpairs_alldata = train_allpairs_alldata[labels].values.ravel()
x_train_allpairs_alldata_means = np.mean(x_train_allpairs_alldata, axis=1).reshape(-1,1)
x_train_allpairs_alldata_stds = np.std(x_train_allpairs_alldata, axis=1).reshape(-1,1)

test_allpairs_alldata = pd.concat([test_alldata[pair] for pair in pairs])
x_test_allpairs_alldata = np.hstack([test_allpairs_alldata[features].values])
y_test_allpairs_alldata = test_allpairs_alldata[labels].values.ravel()

train_allpairs_realdist = pd.concat([train_realdist[pair] for pair in pairs])
x_train_allpairs_realdist = np.hstack([train_allpairs_realdist[features].values])
y_train_allpairs_realdist = train_allpairs_realdist[labels].values.ravel()
x_train_allpairs_realdist_means = np.mean(x_train_allpairs_realdist, axis=1).reshape(-1,1)
x_train_allpairs_realdist_stds = np.std(x_train_allpairs_realdist, axis=1).reshape(-1,1)

test_allpairs_realdist = pd.concat([test_realdist[pair] for pair in pairs])
x_test_allpairs_realdist = np.hstack([test_allpairs_realdist[features].values])
y_test_allpairs_realdist = test_allpairs_realdist[labels].values.ravel()

train_allpairs_balanced = pd.concat([train_balanced[pair] for pair in pairs])
x_train_allpairs_balanced = np.hstack([train_allpairs_balanced[features].values])
y_train_allpairs_balanced = train_allpairs_balanced[labels].values.ravel()
x_train_allpairs_balanced_means = np.mean(x_train_allpairs_balanced, axis=1).reshape(-1,1)
x_train_allpairs_balanced_stds = np.std(x_train_allpairs_balanced, axis=1).reshape(-1,1)

test_allpairs_balanced = pd.concat([test_balanced[pair] for pair in pairs])
x_test_allpairs_balanced = np.hstack([test_allpairs_balanced[features].values])
y_test_allpairs_balanced = test_allpairs_balanced[labels].values.ravel()

{'Hindi-Persian': {'target': {'name': 'Hindi', 'epi': 'hin-Deva', 'gtrans': 'hi', 'unicode': 'Devanagari'}, 'source': {'name': 'Persian', 'epi': 'fas-Arab', 'gtrans': 'fa', 'unicode': 'Arabic'}, 'wiki': 'https://en.m.wiktionary.org/wiki/Category:Hindi_terms_borrowed_from_Persian'}, 'English-French': {'target': {'name': 'English', 'epi': 'eng-Latn', 'gtrans': 'en', 'unicode': 'Latin'}, 'source': {'name': 'French', 'epi': 'fra-Latn', 'gtrans': 'fr', 'unicode': 'Latin'}, 'wiki': 'https://en.m.wiktionary.org/wiki/Category:English_terms_borrowed_from_French'}}
Hindi-Persian
English-French


# Logistic Regression classifier

In [5]:
def lr_evaluate(full_train_set, full_test_set, pairs_list):    
    train_set = pd.concat([full_train_set[pair] for pair in pairs])
    x_train = train_set[features].values
    x_means = np.mean(x_train, axis=0)
    x_stds = np.std(x_train, axis=0)
    y_train = train_set[labels].values.ravel()
    
    test_set = pd.concat([full_test_set[pair] for pair in pairs])
    x_test = test_set[features].values
    y_test = test_set[labels].values.ravel()
    
    # standardize input features
    x_train = (x_train - x_means)/x_stds

    print("Training on all langs\n")
    LR = LogisticRegression(random_state=1, solver='lbfgs', multi_class='ovr', max_iter=500).fit(x_train, y_train)

    # display regressor weights
    df = pd.DataFrame(LR.coef_, columns=features).style.set_caption('Weights')
    display(df)
    print()
    
    print("Evaluating on all langs")
    x_test = (x_test - x_means)/x_stds
    y_pred = LR.predict(x_test)

    print("f1-score : ", f1_score(y_test, y_pred ))
    print("precision : ",precision_score(y_test, y_pred))
    print("recall : ",recall_score(y_test, y_pred )) 
    print("accuracy : ",accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    print()
    
    # display false positives and false negatives
    unq = np.array([x + 2*y for x, y in zip(y_pred, y_test)])
    tp = np.array(np.where(unq == 3)).tolist()[0]
    fp = np.array(np.where(unq == 1)).tolist()[0]
    tn = np.array(np.where(unq == 0)).tolist()[0]
    fn = np.array(np.where(unq == 2)).tolist()[0]

    display(test_set.reset_index(drop=True).iloc[fp,:].style.set_caption('False positives'))
    display(test_set.reset_index(drop=True).iloc[fn,:].style.set_caption('False negatives'))
    
    for pair in pairs_list:
        print(f'Evaluating on {pair}')
        
        test_set = full_test_set[pair]
        x_test = test_set[features].values
        y_test = test_set[labels].values.ravel()
        
        x_test = (x_test - x_means)/x_stds
        y_pred = LR.predict(x_test)
        
        print("f1-score : ", f1_score(y_test, y_pred ))
        print("precision : ",precision_score(y_test, y_pred))
        print("recall : ",recall_score(y_test, y_pred )) 
        print("accuracy : ",accuracy_score(y_test, y_pred))
        print(classification_report(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))
        print()
        
        # display false positives and false negatives
        unq = np.array([x + 2*y for x, y in zip(y_pred, y_test)])
        tp = np.array(np.where(unq == 3)).tolist()[0]
        fp = np.array(np.where(unq == 1)).tolist()[0]
        tn = np.array(np.where(unq == 0)).tolist()[0]
        fn = np.array(np.where(unq == 2)).tolist()[0]

        display(test_set.iloc[fp,:].style.set_caption('False positives'))
        display(test_set.iloc[fn,:].style.set_caption('False negatives'))
        
    for pair in pairs_list:
        print(f'Training and evaluating on {pair}')
        
        train_set = full_train_set[pair]
        x_train = train_set[features].values
        x_means = np.mean(x_train, axis=0)
        x_stds = np.std(x_train, axis=0)
        y_train = train_set[labels].values.ravel()
        
        test_set = full_test_set[pair]
        x_test = test_set[features].values
        y_test = test_set[labels].values.ravel()
        
        x_train = (x_train - x_means)/x_stds
        
        LR = LogisticRegression(random_state=1, solver='lbfgs', multi_class='ovr', max_iter=500).fit(x_train, y_train)
        
        # display regressor weights
        df = pd.DataFrame(LR.coef_, columns=features).style.set_caption('Weights')
        display(df)

        x_test = (x_test - x_means)/x_stds
        y_pred = LR.predict(x_test)
        
        print("f1-score : ", f1_score(y_test, y_pred ))
        print("precision : ",precision_score(y_test, y_pred))
        print("recall : ",recall_score(y_test, y_pred )) 
        print("accuracy : ",accuracy_score(y_test, y_pred))
        print(classification_report(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))
        print()
        
        # display false positives and false negatives
        unq = np.array([x + 2*y for x, y in zip(y_pred, y_test)])
        tp = np.array(np.where(unq == 3)).tolist()[0]
        fp = np.array(np.where(unq == 1)).tolist()[0]
        tn = np.array(np.where(unq == 0)).tolist()[0]
        fn = np.array(np.where(unq == 2)).tolist()[0]

        display(test_set.iloc[fp,:].style.set_caption('False positives'))
        display(test_set.iloc[fn,:].style.set_caption('False negatives'))

In [6]:
print("Evaluating on alldata splits\n")
lr_evaluate(train_alldata, test_alldata, pairs)

print("Evaluating on realdist splits\n")
lr_evaluate(train_realdist, test_realdist, pairs)

print("Evaluating on balanced splits\n")
lr_evaluate(train_balanced, test_balanced, pairs)

Evaluating on alldata splits

Training on all langs



,Fast Levenshtein Distance Div Maxlen,Dolgo Prime Distance Div Maxlen,Feature Edit Distance Div Maxlen,Hamming Feature Distance Div Maxlen,Weighted Feature Distance Div Maxlen,Partial Hamming Feature Distance Div Maxlen,plain Levenshtein,DNN_logits,MBERT_cos_sim,XLM_cos_sim
0,-0.436158,-0.152060,0.330502,0.234419,0.202898,-0.394573,-0.535563,5.280771,0.648894,0.220875



Evaluating on all langs
f1-score :  0.7360970677451971
precision :  0.7711864406779662
recall :  0.7040618955512572
accuracy :  0.9122689075630253
              precision    recall  f1-score   support

           0       0.94      0.96      0.95      2458
           1       0.77      0.70      0.74       517

    accuracy                           0.91      2975
   macro avg       0.86      0.83      0.84      2975
weighted avg       0.91      0.91      0.91      2975

[[2350  108]
 [ 153  364]]



,Unnamed: 0,Unnamed: 0.1,loan_word,original_word,loan_word_epitran,original_word_epitran,loan_english,original_english,Fast Levenshtein Distance Div Maxlen,Dolgo Prime Distance Div Maxlen,Feature Edit Distance Div Maxlen,Hamming Feature Distance Div Maxlen,Weighted Feature Distance Div Maxlen,Partial Hamming Feature Distance Div Maxlen,plain Levenshtein,label,label_bin,DNN_logits,MBERT_cos_sim,XLM_cos_sim,Unnamed: 0.1.1
61,61,2431,फ़ायदा,فایده,faːjdaː,fɒjdh,advantage,Faydeh,0.571429,0.200000,0.075000,0.083333,1.425000,0.075000,6,hard_negative,0,1.957343,0.607233,0.632149,nan
75,75,705,ख़त,خفت,xət,xft,letter,faint,0.333333,0.333333,0.104167,0.125000,2.583333,0.104167,3,hard_negative,0,0.627898,0.568654,0.723995,nan
215,215,3379,रिश्वत,رشوت,riʃvət,rʃvt,bribe,Bribery,0.333333,0.333333,0.298611,0.333333,2.416667,0.333333,6,hard_negative,0,0.926623,0.686041,0.544634,nan
282,282,5678,मौजूदा,موجود,mɔːd͡ʒuːdaː,mvd͡ʒvd,existing,Available,0.545455,0.500000,0.267361,0.305556,4.041667,0.284722,6,synonym,0,2.950336,0.512315,0.531326,nan
348,348,1035,गुज़ारा,گذار,ɡuzaːraː,ɡzɒr,survive,Transition,0.625000,0.333333,0.319444,0.354167,2.666667,0.354167,7,hard_negative,0,0.720412,0.542360,0.724166,nan
436,436,542,क़ानून,کانون,qaːnuːn,kɒnvn,law,Canon,0.714286,0.200000,0.104167,0.116667,2.100000,0.104167,6,hard_negative,0,6.771222,0.565388,0.775422,nan
444,444,1257,ज़कात,ذکاوت,zəkaːt,zkɒvt,Zakat,ذکاوت,0.500000,0.400000,0.195833,0.225000,3.200000,0.195833,5,hard_negative,0,0.976864,0.462078,0.672490,nan
615,615,3448,रोज़ाना,رضوان,rozaːnaː,rzvɒn,everyday,Radwan,0.625000,0.500000,0.243056,0.277778,3.000000,0.260417,7,hard_negative,0,0.657934,0.581792,0.712921,nan
848,848,1432,बर्बर,بربر,bərbər,brbr,barbarian,barbarian,0.333333,0.333333,0.298611,0.333333,2.416667,0.333333,5,synonym,0,4.265605,0.624180,0.600969,nan
959,959,863,ख़ुशामद,خوش‌آمد,xuʃaːməd,xvʃ‌ɒmd,happy,Welcome,0.500000,0.285714,0.190476,0.214286,2.392857,0.205357,7,hard_negative,0,0.766222,0.496819,0.719226,nan


,Unnamed: 0,Unnamed: 0.1,loan_word,original_word,loan_word_epitran,original_word_epitran,loan_english,original_english,Fast Levenshtein Distance Div Maxlen,Dolgo Prime Distance Div Maxlen,Feature Edit Distance Div Maxlen,Hamming Feature Distance Div Maxlen,Weighted Feature Distance Div Maxlen,Partial Hamming Feature Distance Div Maxlen,plain Levenshtein,label,label_bin,DNN_logits,MBERT_cos_sim,XLM_cos_sim,Unnamed: 0.1.1
64,64,79,आशिक़ी,عاشقی,aːʃiqiː,ʔɒʃɣj,Aashiqui,being in love,0.857143,0.600000,0.250000,0.291667,4.125000,0.250000,6,loan,1,-1.864414,0.469438,0.733237,nan
80,80,834,बाक़ायदा,باقاعده,baːqaːjdaː,bɒɣɒʔdh,quite often,Regularly,0.800000,0.142857,0.113095,0.119048,1.767857,0.113095,8,loan,1,-0.342959,0.408781,0.507330,nan
88,88,392,जलालाबाद,جلال‌آباد,d͡ʒəlaːlaːbaːd,d͡ʒlɒl‌ɒbɒd,Jalalabad,Jalalabad,0.500000,0.111111,0.141204,0.152778,1.305556,0.152778,9,loan,1,-7.722263,0.589794,0.724651,nan
98,98,456,जान,جان,d͡ʒaːn,d͡ʒɒn,Life,John,0.333333,0.000000,0.041667,0.041667,0.500000,0.041667,3,loan,1,-1.289959,0.445320,0.774739,nan
148,148,729,पाजामा,پايجامه / پاجامه,paːd͡ʒaːmaː,pɒjd͡ʒɒmh / pɒd͡ʒɒmh,pajamas,Pajamas,0.800000,0.538462,0.512821,0.567308,4.250000,0.567308,16,loan,1,-20.556976,0.564887,0.668052,nan
159,159,1158,शागिर्द,شاگرد,ʃaːɡirdə,ʃɒɡrd,disciple,Student,0.500000,0.285714,0.273810,0.303571,2.285714,0.303571,7,loan,1,0.438255,0.497060,0.583752,nan
194,194,82,आसान,آسان,aːsaːn,ɒsɒn,easy,Easy,0.666667,0.000000,0.062500,0.062500,0.750000,0.062500,4,loan,1,-1.954477,0.620488,0.595963,nan
203,203,1125,वाक़ई,واقعی,vaːqəi,vɒɣʔj,really,Real,0.833333,0.400000,0.116667,0.133333,2.000000,0.116667,5,loan,1,-0.559847,0.387109,0.696390,nan
256,256,6,अंदेशा,اندیشه,ndeʃaː,ɒndjʃh,suspect,Thought,0.666667,0.500000,0.211806,0.243056,2.625000,0.229167,6,loan,1,-3.063483,0.391255,0.710758,nan
273,273,533,ताक़त,طاقت,taːqət,tɒɣt,vigor,طاقت,0.666667,0.200000,0.229167,0.250000,2.100000,0.250000,5,loan,1,0.991477,0.493937,0.387655,nan


...

Training and evaluating on English-French


,Fast Levenshtein Distance Div Maxlen,Dolgo Prime Distance Div Maxlen,Feature Edit Distance Div Maxlen,Hamming Feature Distance Div Maxlen,Weighted Feature Distance Div Maxlen,Partial Hamming Feature Distance Div Maxlen,plain Levenshtein,DNN_logits,MBERT_cos_sim,XLM_cos_sim
0,-0.674053,0.560808,-0.707315,1.442646,-0.608890,-0.565429,-0.837315,3.293219,0.503630,0.244202


f1-score :  0.9119318181818181
precision :  0.9968944099378882
recall :  0.8403141361256544
accuracy :  0.8577981651376146
              precision    recall  f1-score   support

           0       0.46      0.98      0.63        54
           1       1.00      0.84      0.91       382

    accuracy                           0.86       436
   macro avg       0.73      0.91      0.77       436
weighted avg       0.93      0.86      0.88       436

[[ 53   1]
 [ 61 321]]



,Unnamed: 0,Unnamed: 0.1,loan_word,original_word,loan_word_epitran,original_word_epitran,loan_english,original_english,Fast Levenshtein Distance Div Maxlen,Dolgo Prime Distance Div Maxlen,Feature Edit Distance Div Maxlen,Hamming Feature Distance Div Maxlen,Weighted Feature Distance Div Maxlen,Partial Hamming Feature Distance Div Maxlen,plain Levenshtein,label,label_bin,Unnamed: 0.1.1,DNN_logits,MBERT_cos_sim,XLM_cos_sim
369,369,1473,espalier,espalière,ɛˈspæljər,ɛspaljə̀ʀ,espalier,espalier,0.444444,0.000000,0.020833,0.031250,0.234375,0.020833,2,hard_negative,0,nan,-0.153409,0.844079,0.692612


,Unnamed: 0,Unnamed: 0.1,loan_word,original_word,loan_word_epitran,original_word_epitran,loan_english,original_english,Fast Levenshtein Distance Div Maxlen,Dolgo Prime Distance Div Maxlen,Feature Edit Distance Div Maxlen,Hamming Feature Distance Div Maxlen,Weighted Feature Distance Div Maxlen,Partial Hamming Feature Distance Div Maxlen,plain Levenshtein,label,label_bin,Unnamed: 0.1.1,DNN_logits,MBERT_cos_sim,XLM_cos_sim
3,3,2217,impermeability,imperméabilité,impermeability*,ɛ̃pɛʀmeabilite,impermeability,impermeability,0.400000,0.071429,0.096726,0.110119,0.866071,0.104167,2,loan,1,nan,-14.115271,0.802988,0.477161
9,9,986,cinema,cinéma,ˈsɪnəmə,sinema,cinema,movie theater,0.571429,0.000000,0.041667,0.041667,0.375000,0.041667,1,loan,1,nan,-7.539831,0.865880,0.836437
15,15,1296,cyme,cime,cyme*,sim,cyme,tops,0.800000,0.500000,0.291667,0.322917,2.718750,0.317708,1,loan,1,nan,-3.919738,0.660974,0.826490
19,19,46,accolé,not comparable,accolé*é,nɔt kɔ̃paʀabl,attached,not comparable,1.000000,0.727273,0.399621,0.439394,3.590909,0.435606,11,loan,1,nan,-4.059823,0.466399,0.495811
21,21,2680,musique concrète,musique concrète,musique* concrète*,myzikə kɔ̃kʀə̀t,concrete music,concrete music,0.833333,0.428571,0.222222,0.247222,1.875000,0.241667,0,loan,1,nan,-9.241386,1.000000,1.000000
22,22,1609,emetic,émétique,ɪˈmɛtɪk,emetik,emetic,emetic,0.571429,0.000000,0.027778,0.027778,0.208333,0.027778,5,loan,1,nan,-3.677054,0.523215,0.745417
24,24,1360,degenerescence,dégénérescence,degenerescence*,deʒeneʀəsɑ̃s,degeneration,degeneration,0.600000,0.384615,0.259615,0.291667,2.769231,0.274038,3,loan,1,nan,-7.066214,0.740021,0.785161
25,25,3145,quatre quart,quatre-quarts,quatre* kwɔrt,katʀə-kaʀ,four quarters,four-quarters,0.769231,0.272727,0.303030,0.344697,2.590909,0.329545,2,loan,1,nan,-7.247688,0.775167,0.537955
27,27,2219,impersonal,impersonnel,ˌɪmˈpərsənəl,ɛ̃pɛʀsɔnəl,impersonal,impersonal,0.583333,0.100000,0.122917,0.141667,1.112500,0.133333,2,loan,1,nan,-5.260839,0.939827,0.869831
37,37,3457,somnambulist,somnambuliste,somnambulist*,sɔnɑ̃bylist,somnambulist,somnambulist,0.461538,0.166667,0.164931,0.184028,1.375000,0.182292,1,loan,1,nan,-7.880839,0.957998,0.694461


# Neural Network classifier

## Definition

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, n_features):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(n_features, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(128, 1),
            
        )

    def forward(self, x):
        logits_new = self.linear_relu_stack(x)
        logits  = logits_new
        
        return torch.sigmoid(logits), logits_new
    
    def fit(self, X_train, Y_train, X_val, Y_val, criterion, optimizer, n_epochs=5000):
        train_losses = []
        val_losses = []
        train_accur = []
        val_accur = []

        for epoch in range(n_epochs):
            y_pred, logits = self(X_train.float())

            train_loss = criterion(y_pred, Y_train.float())

            if epoch % (n_epochs // 50) == 0:
                train_acc,_ = self.calculate_accuracy(Y_train, y_pred)

                y_val_pred = self(X_val.float())[0]

                val_loss = criterion(y_val_pred, Y_val.float())

                val_acc, total_corr = self.calculate_accuracy(Y_val, y_val_pred)

                print(f'''epoch {epoch}
                    Train set - loss: {self.round_tensor(train_loss)}, accuracy: {self.round_tensor(train_acc)} 
                    Val set - loss: {self.round_tensor(val_loss)}, accuracy: {self.round_tensor(val_acc)}''')
                
                train_losses.append(train_loss.detach().cpu().numpy())
                val_losses.append(val_loss.detach().cpu().numpy())

                val_accur.append(val_acc.detach().cpu().numpy())
                train_accur.append(train_acc.detach().cpu().numpy())

            optimizer.zero_grad()

            train_loss.backward()

            optimizer.step()
            
        return train_losses,val_losses,train_accur,val_accur
    
    def calculate_accuracy(self, y_true, y_pred):
        predicted = y_pred.ge(.5) 
        return ((y_true == predicted).sum().float() / len(y_true), (y_true == predicted).sum())
    
    def round_tensor(self, t, decimal_places=3):
        return round(t.item(), decimal_places)
    
    def plot_losses(self, train_losses, val_losses, train_accur, val_accur):
        epochs = range(1, len(train_accur) + 1)

        plt.plot(epochs, train_accur, 'bo', label='Training acc')
        plt.plot(epochs, val_accur, 'b', label='Vaidation acc')
        plt.title('Training and validation accuracy')
        plt.legend()

        plt.figure()

        plt.plot(epochs, train_losses, 'bo', label='Training loss')
        plt.plot(epochs, val_losses, 'b', label='Validation loss')
        plt.title('Training and validation loss')
        plt.legend()

        plt.show()

In [ ]:
def nn_evaluate(full_train_set, full_test_set, pairs_list):    
    train_set = pd.concat([full_train_set[pair] for pair in pairs])
    x_train = train_set[features].values
    x_means = np.mean(x_train, axis=0)
    x_stds = np.std(x_train, axis=0)
    y_train = train_set[labels].values.ravel()
    
    test_set = pd.concat([full_test_set[pair] for pair in pairs])
    x_test = test_set[features].values
    y_test = test_set[labels].values.ravel()
    
    # standardize input features
    x_train = (x_train - x_means)/x_stds
    
    torch.manual_seed(7)
    random.seed(7)
    np.random.seed(7)
    
    model = NeuralNetwork(x_train.shape[1]).to(device)
    print(model)
    
    criterion = nn.BCELoss().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.00001)
    #optimizer = optim.SGD(model.parameters(),lr=0.00001, momentum=0.0,  weight_decay=0.0, nesterov=False)
    
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=1, stratify=y_train)

    x_train = torch.tensor(x_train).to(device)
    x_val = torch.tensor(x_val).to(device)
    y_train = torch.tensor(y_train).reshape(-1,1).to(device)
    y_val = torch.tensor(y_val).reshape(-1,1).to(device)
        
    print("Training on all langs\n")
    train_losses, val_losses, train_accur, val_accur = \
        model.fit(x_train, y_train, x_val, y_val, criterion, optimizer, n_epochs=5000)
    model.plot_losses(train_losses,val_losses,train_accur,val_accur)
    
    model.eval()
    
    x_test = (x_test - x_means)/x_stds
    x_test = torch.tensor(x_test).to(device)

    with torch.no_grad():
        y_pred = model(torch.tensor(x_test).float())[0] > .5
        y_pred = y_pred.detach().cpu().numpy()

        print("f1-score : ", f1_score(y_test, y_pred))
        print("precision : ",precision_score(y_test, y_pred))
        print("recall : ",recall_score(y_test, y_pred)) 
        print("accuracy : ",accuracy_score(y_test, y_pred))
        print(classification_report(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))
        print()
        
        # display false positives and false negatives
        unq = np.array([x + 2*y for x, y in zip(y_pred, y_test)])
        tp = np.array(np.where(unq == 3)).tolist()[0]
        fp = np.array(np.where(unq == 1)).tolist()[0]
        tn = np.array(np.where(unq == 0)).tolist()[0]
        fn = np.array(np.where(unq == 2)).tolist()[0]

        #display(test_set.iloc[fp,:].style.set_caption('False positives'))
        display(test_set.reset_index(drop=True).iloc[fp,:].style.set_caption('False positives'))
        display(test_set.reset_index(drop=True).iloc[fn,:].style.set_caption('False negatives'))
        #display(test_set.iloc[fn,:].style.set_caption('False negatives'))
        
        for pair in pairs_list:
            print(f'Evaluating on {pair}')

            test_set = full_test_set[pair]
            x_test = test_set[features].values
            y_test = test_set[labels].values.ravel()

            x_test = (x_test - x_means)/x_stds
            x_test = torch.tensor(x_test).to(device)
            y_pred = model(torch.tensor(x_test).float())[0] > .5
            y_pred = y_pred.detach().cpu().numpy()
            
 
            print("f1-score : ", f1_score(y_test, y_pred ))
            print("precision : ",precision_score(y_test, y_pred))
            print("recall : ",recall_score(y_test, y_pred )) 
            print("accuracy : ",accuracy_score(y_test, y_pred))
            print(classification_report(y_test, y_pred))
            print(confusion_matrix(y_test, y_pred))
            print()
            
            # display false positives and false negatives
            unq = np.array([x + 2*y for x, y in zip(y_pred, y_test)])
            tp = np.array(np.where(unq == 3)).tolist()[0]
            fp = np.array(np.where(unq == 1)).tolist()[0]
            tn = np.array(np.where(unq == 0)).tolist()[0]
            fn = np.array(np.where(unq == 2)).tolist()[0]

            display(test_set.iloc[fp,:].style.set_caption('False positives'))
            display(test_set.iloc[fn,:].style.set_caption('False negatives'))
            
    for pair in pairs_list:
        print(f'Training and evaluating on {pair}')
        
        train_set = full_train_set[pair]
        x_train = train_set[features].values
        x_means = np.mean(x_train, axis=0)
        x_stds = np.std(x_train, axis=0)
        y_train = train_set[labels].values.ravel()
        
        test_set = full_test_set[pair]
        x_test = test_set[features].values
        y_test = test_set[labels].values.ravel()
        
        x_train = (x_train - x_means)/x_stds
        
        model = NeuralNetwork(x_train.shape[1]).to(device)
    
        criterion = nn.BCELoss().to(device)
        optimizer = optim.Adam(model.parameters(), lr=0.00001)

        x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=1, stratify=y_train)

        x_train = torch.tensor(x_train).to(device)
        x_val = torch.tensor(x_val).to(device)
        y_train = torch.tensor(y_train).reshape(-1,1).to(device)
        y_val = torch.tensor(y_val).reshape(-1,1).to(device)

        train_losses, val_losses, train_accur, val_accur = \
            model.fit(x_train, y_train, x_val, y_val, criterion, optimizer, n_epochs=5000)
        model.plot_losses(train_losses,val_losses,train_accur,val_accur)

        model.eval()

        x_test = (x_test - x_means)/x_stds
        x_test = torch.tensor(x_test).to(device)
        
        with torch.no_grad():
            y_pred = model(torch.tensor(x_test).float())[0] > .5
            y_pred = y_pred.detach().cpu().numpy()

            print("f1-score : ", f1_score(y_test, y_pred ))
            print("precision : ",precision_score(y_test, y_pred))
            print("recall : ",recall_score(y_test, y_pred )) 
            print("accuracy : ",accuracy_score(y_test, y_pred))
            print(classification_report(y_test, y_pred))
            print(confusion_matrix(y_test, y_pred))
            print()
            
            # display false positives and false negatives
            unq = np.array([x + 2*y for x, y in zip(y_pred, y_test)])
            tp = np.array(np.where(unq == 3)).tolist()[0]
            fp = np.array(np.where(unq == 1)).tolist()[0]
            tn = np.array(np.where(unq == 0)).tolist()[0]
            fn = np.array(np.where(unq == 2)).tolist()[0]

            display(test_set.iloc[fp,:].style.set_caption('False positives'))
            display(test_set.iloc[fn,:].style.set_caption('False negatives'))

In [ ]:
print("Evaluating on alldata splits\n")
nn_evaluate(train_alldata, test_alldata, pairs)

print("Evaluating on realdist splits\n")
nn_evaluate(train_realdist, test_realdist, pairs)

print("Evaluating on balanced splits\n")
nn_evaluate(train_balanced, test_balanced, pairs)

# SVM Classifier

In [ ]:
def svm_evaluate(full_train_set, full_test_set, pairs_list):    
    train_set = pd.concat([full_train_set[pair] for pair in pairs])
    x_train = train_set[features].values
    x_means = np.mean(x_train, axis=0)
    x_stds = np.std(x_train, axis=0)
    y_train = train_set[labels].values.ravel()
    
    test_set = pd.concat([full_test_set[pair] for pair in pairs])
    x_test = test_set[features].values
    y_test = test_set[labels].values.ravel()
    
    # standardize input features
    x_train = (x_train - x_means)/x_stds

    print("Training on all langs\n")
     
    SVM = SVC(kernel='linear')
    SVM.fit(x_train, y_train)

    # display regressor weights
    df = pd.DataFrame(SVM.coef_, columns=features).style.set_caption('Weights')
    display(df)
    print()
    
    print("Evaluating on all langs")
    x_test = (x_test - x_means)/x_stds
    y_pred = SVM.predict(x_test)

    print("f1-score : ", f1_score(y_test, y_pred ))
    print("precision : ",precision_score(y_test, y_pred))
    print("recall : ",recall_score(y_test, y_pred )) 
    print("accuracy : ",accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    print()
    
    # display false positives and false negatives
    unq = np.array([x + 2*y for x, y in zip(y_pred, y_test)])
    tp = np.array(np.where(unq == 3)).tolist()[0]
    fp = np.array(np.where(unq == 1)).tolist()[0]
    tn = np.array(np.where(unq == 0)).tolist()[0]
    fn = np.array(np.where(unq == 2)).tolist()[0]

    display(test_set.reset_index(drop=True).iloc[fp,:].style.set_caption('False positives'))
    display(test_set.reset_index(drop=True).iloc[fn,:].style.set_caption('False negatives'))
    
    for pair in pairs_list:
        print(f'Evaluating on {pair}')
        
        test_set = full_test_set[pair]
        x_test = test_set[features].values
        y_test = test_set[labels].values.ravel()
        
        x_test = (x_test - x_means)/x_stds
        y_pred = SVM.predict(x_test)
        
        print("f1-score : ", f1_score(y_test, y_pred ))
        print("precision : ",precision_score(y_test, y_pred))
        print("recall : ",recall_score(y_test, y_pred )) 
        print("accuracy : ",accuracy_score(y_test, y_pred))
        print(classification_report(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))
        print()
        
        # display false positives and false negatives
        unq = np.array([x + 2*y for x, y in zip(y_pred, y_test)])
        tp = np.array(np.where(unq == 3)).tolist()[0]
        fp = np.array(np.where(unq == 1)).tolist()[0]
        tn = np.array(np.where(unq == 0)).tolist()[0]
        fn = np.array(np.where(unq == 2)).tolist()[0]

        display(test_set.iloc[fp,:].style.set_caption('False positives'))
        display(test_set.iloc[fn,:].style.set_caption('False negatives'))
        
    for pair in pairs_list:
        print(f'Training and evaluating on {pair}')
        
        train_set = full_train_set[pair]
        x_train = train_set[features].values
        x_means = np.mean(x_train, axis=0)
        x_stds = np.std(x_train, axis=0)
        y_train = train_set[labels].values.ravel()
        
        test_set = full_test_set[pair]
        x_test = test_set[features].values
        y_test = test_set[labels].values.ravel()
        
        x_train = (x_train - x_means)/x_stds
        
        SVM = SVC(kernel='linear')
        SVM.fit(x_train, y_train)
        
        # display regressor weights
        df = pd.DataFrame(SVM.coef_, columns=features).style.set_caption('Weights')
        display(df)

        x_test = (x_test - x_means)/x_stds
        y_pred = SVM.predict(x_test)
        
        print("f1-score : ", f1_score(y_test, y_pred ))
        print("precision : ",precision_score(y_test, y_pred))
        print("recall : ",recall_score(y_test, y_pred )) 
        print("accuracy : ",accuracy_score(y_test, y_pred))
        print(classification_report(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))
        print()
        
        # display false positives and false negatives
        unq = np.array([x + 2*y for x, y in zip(y_pred, y_test)])
        tp = np.array(np.where(unq == 3)).tolist()[0]
        fp = np.array(np.where(unq == 1)).tolist()[0]
        tn = np.array(np.where(unq == 0)).tolist()[0]
        fn = np.array(np.where(unq == 2)).tolist()[0]

        display(test_set.iloc[fp,:].style.set_caption('False positives'))
        display(test_set.iloc[fn,:].style.set_caption('False negatives'))

In [ ]:
print("Evaluating on alldata splits\n")
svm_evaluate(train_alldata, test_alldata, pairs)

print("Evaluating on realdist splits\n")
svm_evaluate(train_realdist, test_realdist, pairs)

print("Evaluating on balanced splits\n")
svm_evaluate(train_balanced, test_balanced, pairs)

# Random Forest Classifier 

In [ ]:
def rf_evaluate(full_train_set, full_test_set, pairs_list):    
    train_set = pd.concat([full_train_set[pair] for pair in pairs])
    x_train = train_set[features].values
    x_means = np.mean(x_train, axis=0)
    x_stds = np.std(x_train, axis=0)
    y_train = train_set[labels].values.ravel()
    
    test_set = pd.concat([full_test_set[pair] for pair in pairs])
    x_test = test_set[features].values
    y_test = test_set[labels].values.ravel()
    
    # standardize input features
    x_train = (x_train - x_means)/x_stds

    print("Training on all langs\n")
     
    RF = RandomForestClassifier(n_estimators=1000, max_depth=5, random_state=0)
    RF.fit(x_train, y_train)
#     num_leafs = [1, 5, 10, 20, 50, 100]
#     parameters3 = [{'n_estimators' : range(100,800,20),
#              'max_depth': range(1,20,5),
#              'min_samples_leaf':num_leafs
#              }]
    
#     gs3 = GridSearchCV(estimator=RF,
#                   param_grid=parameters3,
#                   cv = 10,
#                   n_jobs = -1)

#     gs3.fit(x_train, y_train)
    
    # display regressor weights
    #df = pd.DataFrame(RF.best_params_, columns=features).style.set_caption('Weights')
    #display(df)
    print()
    
    print("Evaluating on all langs")
    x_test = (x_test - x_means)/x_stds
    y_pred = RF.predict(x_test)

    print("f1-score : ", f1_score(y_test, y_pred ))
    print("precision : ",precision_score(y_test, y_pred))
    print("recall : ",recall_score(y_test, y_pred )) 
    print("accuracy : ",accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    print()
    
    # display false positives and false negatives
    unq = np.array([x + 2*y for x, y in zip(y_pred, y_test)])
    tp = np.array(np.where(unq == 3)).tolist()[0]
    fp = np.array(np.where(unq == 1)).tolist()[0]
    tn = np.array(np.where(unq == 0)).tolist()[0]
    fn = np.array(np.where(unq == 2)).tolist()[0]

    display(test_set.reset_index(drop=True).iloc[fp,:].style.set_caption('False positives'))
    display(test_set.reset_index(drop=True).iloc[fn,:].style.set_caption('False negatives'))
    
    for pair in pairs_list:
        print(f'Evaluating on {pair}')
        
        test_set = full_test_set[pair]
        x_test = test_set[features].values
        y_test = test_set[labels].values.ravel()
        
        x_test = (x_test - x_means)/x_stds
        y_pred = RF.predict(x_test)
        
        print("f1-score : ", f1_score(y_test, y_pred ))
        print("precision : ",precision_score(y_test, y_pred))
        print("recall : ",recall_score(y_test, y_pred )) 
        print("accuracy : ",accuracy_score(y_test, y_pred))
        print(classification_report(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))
        print()
        
        # display false positives and false negatives
        unq = np.array([x + 2*y for x, y in zip(y_pred, y_test)])
        tp = np.array(np.where(unq == 3)).tolist()[0]
        fp = np.array(np.where(unq == 1)).tolist()[0]
        tn = np.array(np.where(unq == 0)).tolist()[0]
        fn = np.array(np.where(unq == 2)).tolist()[0]

        display(test_set.iloc[fp,:].style.set_caption('False positives'))
        display(test_set.iloc[fn,:].style.set_caption('False negatives'))
        
    for pair in pairs_list:
        print(f'Training and evaluating on {pair}')
        
        train_set = full_train_set[pair]
        x_train = train_set[features].values
        x_means = np.mean(x_train, axis=0)
        x_stds = np.std(x_train, axis=0)
        y_train = train_set[labels].values.ravel()
        
        test_set = full_test_set[pair]
        x_test = test_set[features].values
        y_test = test_set[labels].values.ravel()
        
        x_train = (x_train - x_means)/x_stds
        
        RF = RandomForestClassifier(n_estimators=1000, max_depth=5, random_state=0)
        RF.fit(x_train, y_train)
#         num_leafs = [1, 5, 10, 20, 50, 100]
#         parameters3 = [{'n_estimators' : range(100,800,20),
#              'max_depth': range(1,20,5),
#              'min_samples_leaf':num_leafs
#              }]
    
#         gs3 = GridSearchCV(estimator=RF,
#                   param_grid=parameters3,
#                   cv = 10,
#                   n_jobs = -1)

#         gs3.fit(x_train, y_train)
        
        # display regressor weights
        #df = pd.DataFrame(RF.best_params_, columns=features).style.set_caption('Weights')
        #display(df)

        x_test = (x_test - x_means)/x_stds
        y_pred = RF.predict(x_test)
        
        print("f1-score : ", f1_score(y_test, y_pred ))
        print("precision : ",precision_score(y_test, y_pred))
        print("recall : ",recall_score(y_test, y_pred )) 
        print("accuracy : ",accuracy_score(y_test, y_pred))
        print(classification_report(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))
        print()
        
        # display false positives and false negatives
        unq = np.array([x + 2*y for x, y in zip(y_pred, y_test)])
        tp = np.array(np.where(unq == 3)).tolist()[0]
        fp = np.array(np.where(unq == 1)).tolist()[0]
        tn = np.array(np.where(unq == 0)).tolist()[0]
        fn = np.array(np.where(unq == 2)).tolist()[0]

        display(test_set.iloc[fp,:].style.set_caption('False positives'))
        display(test_set.iloc[fn,:].style.set_caption('False negatives'))

In [ ]:
print("Evaluating on alldata splits\n")
rf_evaluate(train_alldata, test_alldata, pairs)

print("Evaluating on realdist splits\n")
rf_evaluate(train_realdist, test_realdist, pairs)

print("Evaluating on balanced splits\n")
rf_evaluate(train_balanced, test_balanced, pairs)